In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn as skl
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
import torch
import torchvision.transforms as transforms
import os
import pytorch_lightning as pl
from PIL import Image
from IPython.display import Image as Iimage
from matplotlib.pyplot import imshow
import json
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision import models
import time
import copy
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

C:\Users\99298\anaconda3\envs\ai\lib\site-packages\pytorch_lightning\metrics\__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


# File paths

In [2]:
train_image_path = 'images/'
image_path = 'test1/images/'
lb_path = 'LB/images/'
# img = Image.fromarray(img.astype('uint8')).convert('RGB') # convert array to image

# Load data and labels

In [3]:
label_df = pd.read_csv('test1/' + 'test_labels.csv',index_col='filename')
label_df.head()

,label
filename,
hc00011440.jpg,cc
hc00021440.jpg,cc
hc00031440.jpg,cc
hc00041440.jpg,cc
hc00051440.jpg,cc


In [4]:
label_df.loc['hc00031440.jpg','label']

'cc'

In [5]:
label_df['label'].unique()

array(['cc', 'jhc', 'ptcl'], dtype=object)

In [6]:
type_dict = {'cc':(1,0,0),'jhc':(0,1,0),'ptcl':(0,0,1)}
type_list = ['cc','jhc','ptcl']

# Define picture transformation and augmentation
# <font color='red'> change color code </font>

In [7]:
resize = transforms.Resize(224)
resize256 = transforms.Resize((256,256))
blur = transforms.GaussianBlur(kernel_size = (3,3))
#equalizer = transforms.RandomEqualize()
autoaugment = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees = 10,translate = (0.06,0.06),scale=(0.9,1))
])
totensor = transforms.ToTensor()
tenCrop = transforms.TenCrop(size=(224,224))
randomCrop = transforms.RandomCrop(size=(224,224))
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [8]:
image = Image.open(image_path+'hc03601440.jpg')
#image.show()
crop_size = image.size[0] if image.size[0]< image.size[1] else image.size[1]
image = autoaugment(image)
image = transforms.CenterCrop(crop_size)(image)
image = resize(image)
image.show()

FileNotFoundError: [Errno 2] No such file or directory: 'test1/images/hc03601440.jpg'

# Load images

In [9]:
allimgs = []
labels = []
for file in os.listdir(image_path):
    image = Image.open(image_path + file)
    one = [image]
    label = label_df.loc[file,'label']
    crop_size = image.size[0] if image.size[0]< image.size[1] else image.size[1]
    one.append(crop_size)
    labels.append(type_dict[label])
    allimgs.append(one)
labels = torch.tensor(labels)

In [10]:
Y = np.zeros(570)
for i in range(110):
    Y[80 + i] = 1
for i in range(380):
    Y[190 + i] = 2

In [11]:
pred_images = []
pred_images_names = []
for file in os.listdir(lb_path):
    image = Image.open(lb_path + file)
    crop_size = image.size[0] if image.size[0]< image.size[1] else image.size[1]
    image = transforms.CenterCrop(crop_size)(image)
    image = resize(image)
    pred_images.append(image)
    pred_images_names.append(file)

In [164]:
preds = []
for file in os.listdir(lb_path):
    image = Image.open(lb_path + file)
    image = resize256(image)
    preds.append(image)

In [138]:
test_images = []
for file in os.listdir(lb_path):
    image = Image.open(lb_path + file)
    test_images.append(image)


# DataLoader

In [266]:
class Augmented_dataset(Dataset):
    def __init__(self, imgs,labels):
        self.imgs = imgs
        self.labels = labels
        
    def __len__(self):
        # RETURN SIZE OF DATASET
        length = len(self.labels) 
        return length

    def __getitem__(self, idx):
        # RETURN IMAGE AT GIVEN idx
        image = self.imgs[idx][0]
        crop_size = self.imgs[idx][1]
        if np.random.uniform(0,1)>0.5:
            image = blur(image)
        if np.random.uniform(0,1)>0.01:
            image = autoaugment(image)
        image = resize256(image)
        image = randomCrop(image)
        image = totensor(image)
        image = normalize(image)
        return image,torch.argmax(self.labels[idx])
    
class Val_dataset(Dataset):
    def __init__(self, imgs,labels):
        self.imgs = imgs
        self.labels = labels
        
    def __len__(self):
        # RETURN SIZE OF DATASET
        length = len(self.labels) 
        return length

    def __getitem__(self, idx):
        # RETURN IMAGE AT GIVEN idx
        image = totensor(self.imgs[idx])
        image = normalize(image)
        return image,self.labels[idx]

In [267]:
train_ds = Augmented_dataset(allimgs,labels)
val_ds = Val_dataset(pred_images,Y)
batch_size = 2
train_dl = DataLoader(train_ds,batch_size=batch_size,shuffle=True)
val_dl = DataLoader(val_ds,batch_size = 570,shuffle=False)

In [268]:
class CNN(pl.LightningModule):
    def __init__(self,loss_func,lr = 1e-3):
        super().__init__()
        self.loss_func = loss_func
        self.learning_rate = lr
        self.conv1 = nn.Sequential( 
            nn.Conv2d(3, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 512, kernel_size=3, stride=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout())
        self.fc1 = nn.Linear(12800, 1000)
        self.fc2 = nn.Linear(1000,3)
    
    def forward(self, x):
        # 
        # x : batch_size x 3 x 224 x 224
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out) 
        out = self.conv4(out) 
        out = self.conv5(out) 
        out = out.view(x.shape[0], out.shape[1]*out.shape[2]*out.shape[3])
        linearOut = F.relu(self.fc1(out))
        #linearOut = torch.sigmoid(self.fc2(linearOut))
        linearOut = self.fc2(linearOut)
        return linearOut
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self(x)
        loss = self.loss_func(y_hat,y)
        
        return {'loss':loss,'y':y,'y_hat':y_hat}
    
    def training_epoch_end(self, training_step_outputs):
        y = torch.tensor([]).to(device='cuda')
        y_hat = torch.tensor([]).to(device='cuda')
        for i in training_step_outputs:
            y = torch.cat((y,i['y']))
            y_hat = torch.cat((y_hat,torch.argmax(i['y_hat'],axis=1)))
        f1 = f1_score(y.cpu().detach().numpy(),y_hat.cpu().detach().numpy(),average='micro')
        print('micro f1 score:',f1)
        print('macro f1 score:',f1_score(y.cpu().detach().numpy(),y_hat.cpu().detach().numpy(),average='macro'))       
        self.log('f1_score',f1)
        #training_loss.append(v)   

In [112]:
class ResNetBottleneck(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(ResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNetUnit(nn.Module):
    def __init__(self, amount, in_channel, out_channel):
        super(ResNetUnit, self).__init__()
        self.in_planes = in_channel
        self.layer = self._make_layer(ResNetBottleneck, out_channel, amount, stride=1)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        out = self.layer(x)
        return out    
class DenseNetBottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(DenseNetBottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)
    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out
class DenseNetUnit(nn.Module):
    def __init__(self, k, amount, in_channel, out_channel, max_input_channel):
        super(DenseNetUnit, self).__init__()
        self.out_channel = out_channel
        if in_channel > max_input_channel:
            self.need_conv = True
            self.bn = nn.BatchNorm2d(in_channel)
            self.conv = nn.Conv2d(in_channel, max_input_channel, kernel_size=1, bias=False)
            in_channel = max_input_channel
        self.layer = self._make_dense(in_channel, k, amount)
    def _make_dense(self, nChannels, growthRate, nDenseBlocks):
        layers = []
        for _ in range(int(nDenseBlocks)):
            layers.append(DenseNetBottleneck(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)
    def forward(self, x):
        out = x
        if hasattr(self, 'need_conv'):
            out = self.conv(F.relu(self.bn(out)))
        out = self.layer(out)
        #print(out.size()[1],self.out_channel)
        assert(out.size()[1] == self.out_channel)
        return out

In [250]:
def train_cnn(model,train_dl,early_stop = False,ep=100):
    early_stop_callback = EarlyStopping(
       monitor='f1_score',
        patience = 5,
       mode='max'
    )
    if early_stop:
        trainer = pl.Trainer(max_epochs = ep,gpus = 1,precision = 16,auto_lr_find=True,
                callbacks=[early_stop_callback])
    else:
        trainer = pl.Trainer(max_epochs = ep,gpus = 1,precision = 16,auto_lr_find=True)
                #callbacks=[early_stop_callback])
    trainer.fit(model,train_dl)

def test_cnn(model,pred_images):
    model.eval()
    result = []
    for image in pred_images:
        crop_size = image.size[0] if image.size[0]< image.size[1] else image.size[1]
        image = transforms.CenterCrop(crop_size)(image)
        image = resize(image)
        image = torch.unsqueeze(totensor(image),dim=0)
        out = torch.argmax(model(image)).detach().numpy()
        result.append(int(out))
    return np.asarray(result)

def test_cnn10(model,preds):
    model.eval()
    model.to(device)
    result = []
    for image in preds:
        crop_images = tenCrop(image)
        crop_images = torch.cat([torch.unsqueeze(totensor(i),dim=0) for i in crop_images])
        crop_images.to(device)
        out = model(crop_images)
        out = out.cpu()
        out = torch.argmax(torch.mean(out,dim=0))
        result.append(out)
    return np.asarray(result)
        

def make_result(Y):
    files = os.listdir(lb_path)
    result = []
    for i in range(len(pred_images)):
        one = {}
        one['image_name'] = files[i]
        one['category']   = type_list[int(Y[i])]
        result.append(one)
    with open('results.json', 'w') as fp:
        json.dump(result, fp)

In [275]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in tqdm(range(num_epochs)):

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = torch.tensor(labels,dtype=torch.long).to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

In [269]:
dataloaders_dict = {'train':train_dl,'val':val_dl}

In [276]:
resnet4,resnet_acc_history4 = transfer_learning(resnet4,dataloaders_dict,ep=100)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

C:\Users\99298\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


train Loss: 0.0043 Acc: 1.0000


  1%|▊                                                                                 | 1/100 [00:18<30:21, 18.40s/it]

...

val Loss: 0.2104 Acc: 0.9561

train Loss: 0.0001 Acc: 1.0000


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [36:58<00:22, 22.20s/it]

val Loss: 0.2256 Acc: 0.9456

train Loss: 0.0000 Acc: 1.0000


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [37:20<00:00, 22.41s/it]

val Loss: 0.2102 Acc: 0.9509

Training complete in 37m 21s
Best val Acc: 0.956140


In [277]:
a = test_cnn10(resnet4,preds)
f1_score(Y,a,average='micro')

0.7280701754385965

In [261]:
resnet3,resnet_acc_history3 = transfer_learning(resnet3,dataloaders_dict,ep=100)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Params to learn:
	 fc.weight
	 fc.bias


C:\Users\99298\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


train Loss: 0.5069 Acc: 0.8088


  1%|▊                                                                                 | 1/100 [00:12<20:46, 12.59s/it]

...

val Loss: 0.1547 Acc: 0.9456

train Loss: 0.6367 Acc: 0.7500


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [28:56<00:28, 28.25s/it]

val Loss: 0.1454 Acc: 0.9386

train Loss: 0.5532 Acc: 0.7647


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [29:25<00:00, 17.66s/it]

val Loss: 0.1316 Acc: 0.9509

Training complete in 29m 26s
Best val Acc: 0.964912


In [262]:
resnet3_result = test_cnn10(resnet3,preds)
print(f1_score(Y,resnet3_result,average='micro'))

0.9649122807017544


In [34]:
resnet = initialize_model('resnet',3,True)
resnet1 = initialize_model('resnet',3,True)
resnet2 = initialize_model('resnet',3,True)
resnet3 = initialize_model('resnet',3,True)
resnet4 = initialize_model('resnet',3,False)
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [76]:
def get_train_param(model,feature_extract = True):
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

def transfer_learning(model,dataloaders_dict,ep=25):
    model.to(device)
    optimizer = torch.optim.Adam(get_train_param(model), lr=0.001)
    model,val_acc = train_model(model, dataloaders_dict, nn.CrossEntropyLoss(), 
            optimizer, num_epochs=ep, is_inception=False)
    return model,val_acc

In [77]:
resnet,resnet_acc_history = transfer_learning(resnet,dataloaders_dict,ep=100)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Params to learn:
	 fc.weight
	 fc.bias


C:\Users\99298\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


train Loss: 0.9396 Acc: 0.6029


  1%|▊                                                                                 | 1/100 [00:12<20:40, 12.53s/it]

...

val Loss: 0.1469 Acc: 0.9439

train Loss: 0.4588 Acc: 0.8382


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [21:58<00:13, 13.12s/it]

val Loss: 0.1145 Acc: 0.9579

train Loss: 0.2866 Acc: 0.8971


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [22:11<00:00, 13.32s/it]

val Loss: 0.1374 Acc: 0.9386

Training complete in 22m 12s
Best val Acc: 0.959649


In [54]:
resnet1,resnet_acc_history1 = transfer_learning(resnet1,dataloaders_dict,ep=100)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Params to learn:
	 fc.weight
	 fc.bias


C:\Users\99298\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


train Loss: 1.3781 Acc: 0.4118


  1%|▊                                                                                 | 1/100 [00:12<21:06, 12.79s/it]

...

val Loss: 0.1865 Acc: 0.9193

train Loss: 0.5020 Acc: 0.8235


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [21:55<00:13, 13.40s/it]

val Loss: 0.1191 Acc: 0.9614

train Loss: 0.3237 Acc: 0.8676


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [22:08<00:00, 13.28s/it]

val Loss: 0.1373 Acc: 0.9368

Training complete in 22m 8s
Best val Acc: 0.961404


In [62]:
resnet2,resnet_acc_history2 = transfer_learning(resnet2,dataloaders_dict,ep=100)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Params to learn:
	 fc.weight
	 fc.bias


C:\Users\99298\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


train Loss: 1.4141 Acc: 0.3235


  1%|▊                                                                                 | 1/100 [00:12<21:24, 12.98s/it]

...

val Loss: 0.3585 Acc: 0.8632

train Loss: 0.6477 Acc: 0.7353


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [21:57<00:13, 13.36s/it]

val Loss: 0.1558 Acc: 0.9439

train Loss: 0.2344 Acc: 0.9265


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [22:11<00:00, 13.32s/it]

val Loss: 0.1160 Acc: 0.9526

Training complete in 22m 12s
Best val Acc: 0.961404


In [36]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)
optimizer = torch.optim.Adam(params_to_update, lr=0.001)
resnet,val_acc = train_model(resnet, dataloaders_dict, nn.CrossEntropyLoss(), 
            optimizer, num_epochs=25, is_inception=False)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]C:\Users\99298\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


train Loss: 1.1002 Acc: 0.4559


  4%|███▎                                                                               | 1/25 [00:12<05:07, 12.81s/it]

val Loss: 0.7773 Acc: 0.6912

train Loss: 0.8237 Acc: 0.6471


  8%|██████▋                                                                            | 2/25 [00:25<04:53, 12.75s/it]

...

val Loss: 0.1811 Acc: 0.9211

train Loss: 0.4986 Acc: 0.7941


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [05:18<00:13, 13.30s/it]

val Loss: 0.1753 Acc: 0.9263

train Loss: 0.5191 Acc: 0.8088


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [05:32<00:00, 13.28s/it]

val Loss: 0.1569 Acc: 0.9351

Training complete in 5m 32s
Best val Acc: 0.938596


In [1]:
import numpy as np
a = np.asarray([])
b = np.asarray([1,2,3,4])
np.concatenate((a,b))

array([1., 2., 3., 4.])

In [78]:
resnet_result = test_cnn(resnet,pred_images)
resnet_result

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [39]:
resnet_result1 = test_cnn(resnet1,pred_images)

570


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [63]:
since = time.time()
resnet_result2 = test_cnn(resnet2,pred_images)
print(time.time()-since)
resnet_result2

15.335886001586914


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [83]:
resnet_result2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [92]:
print(f1_score(Y,resnet_result,average='micro'))
print(f1_score(Y,resnet_result1,average='micro'))
print(f1_score(Y,resnet_result2,average='micro'))

0.9596491228070175
0.9614035087719298
0.9614035087719298


In [121]:
aggred = np.round((resnet_result2 + resnet_result +resnet_result2)/3)
print(f1_score(Y,aggred,average='micro'))

0.9631578947368421


In [40]:
f1_score(Y,resnet_result,average='micro')

0.9385964912280702

In [43]:
make_result(resnet_result)

In [122]:
make_result(aggred)

# Save Model

In [60]:
torch.save(resnet1.state_dict(),'resnet1.pth')

# Load model

In [ ]:
resnet1 = initialize_model('resnet',3,True)
resnet1.load_state_dict(torch.load('resnet1.pth'))